Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

A1: The facilities that charge a fee to members are: Tennis Court 1, Tennis Court 2, Massage Room 1, Massage Room 2, and the Squash Court.

Q2: How many facilities do not charge a fee to members?

A2: Four facilities do not charge a fee to members.

Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

A3: The code used is: SELECT  `facid` ,  `name` ,  `membercost` ,  `monthlymaintenance` FROM  `Facilities` WHERE `membercost` < 0.2 *  `monthlymaintenance` AND  `membercost` >0 LIMIT 0 , 30.

Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

A4: The code used is: SELECT * FROM `Facilities` WHERE  `facid` IN ( 1, 5 ) LIMIT 0 , 30

Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? Return the name and monthly maintenance of the facilities in question.

A5: The code used is: SELECT  `name` ,  `monthlymaintenance` , (CASE WHEN  `monthlymaintenance` >100 THEN  'expensive' WHEN  `monthlymaintenance` <=100 THEN  'cheap' END) AS  'categories' FROM  `Facilities` LIMIT 0 , 30

Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

A6: The code used is: SELECT `firstname`,`surname`,`joindate` FROM `Members` WHERE `joindate`=(SELECT MAX(`joindate`) FROM `Members`)

Q7: How can you produce a list of all members who have used a tennis court? Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name.

A7: The code used is: SELECT CONCAT(Facilities.name, ' - ', Members.firstname,' - ', Members.surname) AS 'categories' FROM `Bookings` INNER JOIN `Facilities` ON Bookings.facid = Facilities.facid INNER JOIN `Members` ON Bookings.memid = Members.memid WHERE Facilities.facid IN (0, 1)

Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.

A8: The code used is:

SELECT concat(Facilities.name, ' - GUEST') AS facilities_membername,
Facilities.guestcost * Bookings.slots AS booking_cost
FROM Bookings
JOIN Facilities
ON Bookings.facid = Facilities.facid
Join Members
ON Bookings.memid = Members.memid
WHERE starttime BETWEEN '2012-09-14 00:00:01' AND '2012-09-14 23:59:59'
AND Members.memid = 0

UNION 


SELECT CONCAT(Facilities.name, ' - ',Members.firstname, ' ', Members.surname ) AS facilities_membername,
SUM(Facilities.membercost * Bookings.slots) AS booking_cost
FROM Bookings
JOIN Facilities
ON Bookings.facid = Facilities.facid
JOIN Members
ON Bookings.memid = Members.memid
WHERE starttime BETWEEN '2012-09-14 00:00:01' AND '2012-09-14 23:59:59'
AND Members.memid != 0

ORDER BY booking_cost

Q9: This time, produce the same result as in Q8, but using a subquery.

A9: The code used is:

SELECT concat(guestlist.facname, ' - GUEST') AS facilities_membername,
guestlist.booking_cost
FROM Members


JOIN (

SELECT Bookings.memid AS memid,
Facilities.name AS facname,
slots * guestcost AS booking_cost
FROM Bookings
JOIN Facilities
ON Bookings.facid = Facilities.facid
WHERE starttime BETWEEN '2012-09-14 00:00:01' AND '2012-09-14 23:59:59'
AND memid = 0

) guestlist
ON Members.memid = guestlist.memid
WHERE guestlist.booking_cost > 30


UNION 


SELECT CONCAT(memberslist.facname, ' - ',Members.firstname, ' ', Members.surname ) AS facilities_membername,
memberslist.booking_cost
FROM Members


JOIN (

SELECT Bookings.memid AS memid,
Facilities.name AS facname,
SUM(slots * membercost) AS booking_cost

FROM Bookings
JOIN Facilities
ON Bookings.facid = Facilities.facid
WHERE starttime BETWEEN '2012-09-14 00:00:01' AND '2012-09-14 23:59:59'
AND memid != 0
GROUP BY 1
) memberslist

ON Members.memid = memberslist.memid
WHERE memberslist.booking_cost > 30
ORDER BY booking_cost DESC

Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

A10: The code used is:

SELECT facname, sum(booking_cost) AS booking_cost FROM (

SELECT Facilities.name AS facname,
slots * guestcost AS booking_cost
FROM Bookings
JOIN Facilities
ON Bookings.facid = Facilities.facid
AND memid = 0
GROUP BY facname


UNION 

SELECT Facilities.name AS facname,
SUM(slots * membercost) AS booking_cost

FROM Bookings
JOIN Facilities
ON Bookings.facid = Facilities.facid
AND memid != 0
GROUP BY facname) master_table
GROUP BY facname
HAVING sum(booking_cost) < 1000
ORDER BY booking_cost DESC